In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from datasets import load_dataset


In [7]:
# 1. Load the WikiText-2 dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
raw_text = dataset['train']['text']


In [8]:
# 2. Clean and preprocess text (remove empty lines, lowercase)
corpus = [line.lower() for line in raw_text if line.strip() != ""][:2000]

In [9]:

# 3. Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [10]:
# 4. Create input sequences with sequence length limit
input_sequences = []
max_len_allowed = 20  # Keep sequences short for memory efficiency

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        if len(n_gram_seq) <= max_len_allowed:
            input_sequences.append(n_gram_seq)

In [11]:

# 5. Pad sequences and split into input (X) and label (y)
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [12]:
# 6. Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=64, input_length=max_sequence_len - 1))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 64)            1027648   
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               16500     
                                                                 
 dense (Dense)               (None, 16057)             1621757   
                                                                 
Total params: 2665905 (10.17 MB)
Trainable params: 2665905 (10.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:

# 7. Train the model
model.fit(X, y, epochs=50, batch_size=128, verbose=1)

Epoch 1/50
197/197 [==============================] - 25s 102ms/step - loss: 7.8408 - accuracy: 0.0683
Epoch 2/50
197/197 [==============================] - 20s 102ms/step - loss: 7.1237 - accuracy: 0.0701
Epoch 3/50
197/197 [==============================] - 17s 87ms/step - loss: 6.8634 - accuracy: 0.0866
Epoch 4/50
197/197 [==============================] - 17s 88ms/step - loss: 6.6323 - accuracy: 0.0927
Epoch 5/50
197/197 [==============================] - 22s 109ms/step - loss: 6.4153 - accuracy: 0.0984
Epoch 6/50
197/197 [==============================] - 14s 71ms/step - loss: 6.2017 - accuracy: 0.1074
Epoch 7/50
197/197 [==============================] - 14s 71ms/step - loss: 5.9886 - accuracy: 0.1148
Epoch 8/50
197/197 [==============================] - 14s 73ms/step - loss: 5.7683 - accuracy: 0.1246
Epoch 9/50
197/197 [==============================] - 17s 84ms/step - loss: 5.5502 - accuracy: 0.1369
Epoch 10/50
197/197 [==============================] - 19s 98ms/step - loss: 5.

In [ ]:
def predict_next_word(seed_text, next_words=1):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
def predict_next_word(seed_text, next_words=1):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + predicted_word
    return seed_text


In [23]:
print(predict_next_word("Devan is", 1))
print(predict_next_word("Growing", 1))


Devan is early
Growing season
